## Load in data, clean, and transform

In [160]:
from pandas import Series, DataFrame
import pandas as pd
import glob
import os
import plotly.express as px

In [161]:
list_of_files = glob.glob('/Users/tsbloxsom/Documents/GitHub/Texas-census-county-data-project/Automate collecting of data notebooks/*.csv') 
latest_file = max(list_of_files, key=os.path.getctime)
latest_file.split("\\")[-1]

'Texas county COVID cases data clean.csv'

In [162]:
df = pd.read_csv(latest_file.split("\\")[-1])

In [163]:
df.head()

,County Name,Cases 03-04,Cases 03-05,Cases 03-06,Cases 03-09,Cases 03-10,Cases 03-11,Cases 03-12,Cases 03-13,Cases 03-15,...,Cases 07-31,Cases 08-01,Cases 08-02,Cases 08-03,Cases 08-04,Cases 08-05,Cases 08-06,Cases 08-07,Cases 08-08,Cases 08-09
0,Anderson,0,0,0,0,0,0,0,0,0,...,2196,2209,2209,2307,2329,2355,2353,2379,2402,2402
1,Andrews,0,0,0,0,0,0,0,0,0,...,255,255,266,267,268,273,277,279,297,297
2,Angelina,0,0,0,0,0,0,0,0,0,...,1606,1647,1647,1647,1745,1755,1769,1796,1796,1796
3,Aransas,0,0,0,0,0,0,0,0,0,...,153,153,157,158,158,162,165,169,172,173
4,Archer,0,0,0,0,0,0,0,0,0,...,17,20,20,20,20,20,21,21,21,21


In [164]:
# convert df into time series where rows are each date and clean up
df_t = df.T
df_t.columns = df_t.iloc[0]
df_t = df_t.iloc[1:]
df_t = df_t.iloc[:,:-2]

In [165]:
# next lets convert the index to a date time, must clean up dates first
def clean_index(s):
    s = s.replace("*","")
    s = s[-5:]
    s = s + "-2020"
    #print(s)
    return s

df_t.index = df_t.index.map(clean_index)

df_t.index = pd.to_datetime(df_t.index)

In [166]:
df_t.head()

County Name,Anderson,Andrews,Angelina,Aransas,Archer,Armstrong,Atascosa,Austin,Bailey,Bandera,...,Willacy,Williamson,Wilson,Winkler,Wise,Wood,Yoakum,Young,Zapata,Zavala
2020-03-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-03-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-03-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-03-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-03-10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [167]:
# initalize df with three columns: Date, Case Count, and County
anderson = df_t.T.iloc[0,:]

ts = anderson.to_frame().reset_index()

ts["County"] = "Anderson"
ts = ts.rename(columns = {"Anderson": "Case Count", "index": "Date"})
ts

,Date,Case Count,County
0,2020-03-04,0,Anderson
1,2020-03-05,0,Anderson
2,2020-03-06,0,Anderson
3,2020-03-09,0,Anderson
4,2020-03-10,0,Anderson
...,...,...,...
151,2020-08-05,2355,Anderson
152,2020-08-06,2353,Anderson
153,2020-08-07,2379,Anderson
154,2020-08-08,2402,Anderson


In [168]:
# This while loop adds all counties to the above ts so we can input it into plotly
x = 1
while x < 254:
    new_ts = df_t.T.iloc[x,:]
    new_ts = new_ts.to_frame().reset_index()
    new_ts["County"] = new_ts.columns[1]
    new_ts = new_ts.rename(columns = {new_ts.columns[1]: "Case Count", "index": "Date"})
    ts = pd.concat([ts, new_ts])
    x += 1

In [169]:
ts

,Date,Case Count,County
0,2020-03-04,0,Anderson
1,2020-03-05,0,Anderson
2,2020-03-06,0,Anderson
3,2020-03-09,0,Anderson
4,2020-03-10,0,Anderson
...,...,...,...
151,2020-08-05,197,Zavala
152,2020-08-06,202,Zavala
153,2020-08-07,205,Zavala
154,2020-08-08,210,Zavala


In [171]:
ts.to_csv("time_series_plotly.csv")

## Using plotly to graph time series

In [170]:
fig = px.scatter(ts, x='Date', y='Case Count', color='County')
fig.update_traces(mode='markers+lines')
#fig.update_traces(mode='lines')
fig.show()